In [1]:
from matplotlib import animation
import ray

from run import *

In [2]:
pdb

Automatic pdb calling has been turned ON


2021-08-19 23:34:12,551	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265
2021-08-19 23:34:14,344	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-08-19 23:34:18,881	WARNING util.py:53 -- Install gputil for GPU system monitoring.
2021-08-19 23:34:18,892	INFO trainable.py:377 -- Restored on 192.168.1.233 from checkpoint: /home/anchorwatt/ray_results/ppo5_het/checkpoint_001000/checkpoint-1000
2021-08-19 23:34:18,893	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 1000, '_timesteps_total': None, '_time_total': 7275.971487283707, '_episodes_total': 166660}
2021-08-19 23:34:21,687	WARNING util.py:53 -- Install gputil for GPU system monitoring.
2021-08-19 23:34:21,693	INFO trainable.py:377 -- Restored on 192.168.1.233 from checkpoint: /home/anchorwatt/ray_results/ppo5_het/checkpoint_001000/checkpoint-1000
2021-08-19 23:34:21,694	INFO trainable.py:385 -- Cur

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '432x288', '-pix_fmt', 'rgba', '-r', '5.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', '/home/anchorwatt/ray_results/het-vs-het/video.mp4']' returned non-zero exit status 1.

> /home/anchorwatt/miniconda3/envs/zsc-cleaner/lib/python3.8/site-packages/matplotlib/animation.py(378)_cleanup()
    376                 "MovieWriter stderr:\n%s", err)
    377         if self._proc.returncode:
--> 378             raise subprocess.CalledProcessError(
    379                 self._proc.returncode, self._proc.args, out, err)
    380 



2021-08-19 23:34:33,054	WARNING worker.py:1123 -- The actor or task with ID ffffffffffffffff4c0cad4aa34a3e46175bbe6801000000 cannot be scheduled right now. It requires {CPU: 1.000000} for placement, but this node only has remaining {0.000000/12.000000 CPU, 7.556696 GiB/7.556696 GiB memory, 1.000000/1.000000 GPU, 3.778348 GiB/3.778348 GiB object_store_memory, 1.000000/1.000000 node:192.168.1.233, 1.000000/1.000000 accelerator_type:G}
. In total there are 0 pending tasks and 8 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale or if you specified a runtime_env for this task or actor because it takes time to install.
